## **Visualização e interpretação dos dados**

In [ ]:
# Importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### **Base de Treino - Análise Exploratória**

In [ ]:
# Importando arquivo .csv como um DataFrame pandas
arquivo_treino  =  './dataset/fashion-mnist_train.csv'
train_fmnist = pd.read_csv(arquivo_treino)

In [ ]:
# Obtendo dimensões do DataFrame
train_fmnist.shape

In [ ]:
# Obtendo as primeiras 5 linhas
train_fmnist.head(5)

In [ ]:
# Obtendo se pelo menos uma coluna possui valores nulos
"""  O primeiro any() retorna se há valores nulos em todas as colunas
dada a quantidade de colunas, outro any() é chamado a fim de 
conferir se pelo menos alguma das colunas é nula """
is_null = train_fmnist.isna().any().any()
print("Pelo menos uma coluna possui valores nulos" if is_null else "Nenhuma coluna possui valores nulos" )

### **Visualização dos Dados**

In [ ]:
# Removendo a coluna de label para manter somente os valores dos pixels
pixels = train_fmnist.drop('label', axis=1)
""" pixels.stack() transforma o dataframe em uma série em que todas as colunas vão se condensar
em uma única coluna sendo assim o tamanho da série é dado por 60.000 linhas x 784 colunas
pixels.stack().value_counts() conta a ocorrência de cada valor de intensidade existente 
pixels.stack().value_counts().sort_index() ordena a série em termos dos novo índice, isto é, 
os valores de intensidade de pixel"""
value_counts = pixels.stack().value_counts().sort_index()

plt.bar(value_counts.index, value_counts.values)
plt.xlabel('Valor do Pixel')
plt.ylabel('Frequência')
plt.title('Histograma da Distribuição de Frequência dos Valores dos Pixels')

plt.show()

> - O histograma acima mostra a contagem de ocorrências dos valores de intensidade dos pixels para todas as imagens.  
> - Podemos notar que a maioria dos pixels possuem intensidade 0, isto é, boa parte da região das imagens possuem a cor preta. A cor branca, intensidade 255, parece ser a segunda que mais aparece nas imagens.

### **Distribuição da intensidade média com base no rótulo** 

In [ ]:
# 10 classes de artigos de moda no total
num_classes = 10  
# Preenchendo um array de tamanho 10 com 0s
mean_intensities = np.zeros(num_classes)

# Percorrendo de 0 a 9
for i in range(num_classes):
    # Obtendo as imagens existentes para cada classe
    class_data = train_fmnist[train_fmnist['label'] == i]
    """ Obtendo a média aritmética das intensidades de pixel  
    desconsiderando a primeira coluna (coluna de label) """
    """ Neste caso é obtida a média de todos os valores 
    em todo dataframe, independente da linha e coluna """
    mean_intensity = class_data.iloc[:, 1:].values.mean()
    print(mean_intensity)
    # Atribuindo o valor médio para o índice da classe no vetor
    mean_intensities[i] = mean_intensity

plt.bar(range(num_classes), mean_intensities, color='purple')
plt.title('Distribuição da Intensidade Média por Label')
plt.xlabel('Label')
plt.ylabel('Intensidade Média')
plt.xticks(range(num_classes))
plt.show()

In [ ]:
num_classes = 10
mean_intensities = []

for i in range(num_classes):
    class_data = train_fmnist[train_fmnist['label'] == i]
    """ Neste caso a média é calculada ao longo do eixo 1, 
    o que significa que a média é calculada para cada linha do dataframe 
    gerando um valor médio para cada imagem da classe"""
    mean_intensity = class_data.iloc[:, 1:].values.mean(axis=1)
    print(mean_intensity)
    # Adicionando o vetor com as médias ao vetor geral
    mean_intensities.append(mean_intensity)

label_dictionary = {0: "Camiseta",
                    1: "Calça",
                    2: "Pulôver",
                    3: "Vestido",
                    4: "Casaco",
                    5: "Sandália",
                    6: "Camisa",
                    7: "Tênis",
                    8: "Bolsa",
                    9: "Bota de tornozelo"}

# Criar um gráfico de densidade
plt.figure(figsize=(10, 6))
for i in range(num_classes):
    # sns.kdeplot(mean_intensities[i], label=str(i))
    sns.kdeplot(mean_intensities[i], label=label_dictionary[i])
plt.title('Distribuição da Intensidade Média por Rótulo')
plt.xlabel('Intensidade Média dos Pixels')
plt.ylabel('Densidade')
plt.legend(title='Rótulo')
plt.show()

> - É possível perceber que em geral o padrão das distribuições parecem seguir uma distribuição normal.  
> - Aparentemente tênis e calça parecer ter a uma distribuição normal mais consistente.

In [ ]:
class_variances = []

for i in range(10):
    class_data = train_fmnist[train_fmnist['label'] == i]
    """ A variância é uma medida de dispersão que indica o quão distantes os 
    valores de um conjunto estão da média, neste caso estão sendo analisados 
    todos os valores de pixels para cada classe """
    class_variance = np.var(class_data.iloc[:, 1:].values) 
    class_variances.append(class_variance)

""" Obtendo os índices das classes mais e menos variadas
argmax e arfmin retornam o índice dos maiores e menores valores
calculados, respectivamente """
most_varied_class = np.argmax(class_variances)
least_varied_class = np.argmin(class_variances)

# Configurando 1 figura com 2 subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Plotando histograma da classe menos variada
least_varied_data = train_fmnist[train_fmnist['label'] == least_varied_class].iloc[:, 1:].mean(axis=1)
axs[0].hist(least_varied_data, bins=50, color='blueviolet', alpha=0.7)
axs[0].set_title(f'Histograma da Classe Menos Variada ({label_dictionary[least_varied_class]})')
axs[0].set_xlabel('Intensidade Média de Pixel')
axs[0].set_ylabel('Frequência')

# Plotando histograma da classe mais variada
most_varied_data = train_fmnist[train_fmnist['label'] == most_varied_class].iloc[:, 1:].mean(axis=1)
axs[1].hist(most_varied_data, bins=50, color='indigo', alpha=0.7)
axs[1].set_title(f'Histograma da Classe Mais Variada ({label_dictionary[most_varied_class]})')
axs[1].set_xlabel('Intensidade Média de Pixel')
axs[1].set_ylabel('Frequência')

# Ajustando o layout para evitar sobreposição
plt.tight_layout()
plt.show()

> - O histograma da classe Sandália mostra que os valores dos pixels estão mais concentrados em torno da média, enquanto o histograma de Casaco demonstra que os valores estão mais dispersos em torno da média.  
> - A presença de caudasmais longas no histograma dois certifica a presença de algunsvalores de pixels considerados outliers em relação ao conjunto de dados.

### **Criando e Visualizando Exemplos de Imagens no formato PGM ASCII**  
As imagens PGM (Portable Gray Map) ASCII são um tipo de formato de arquivo de imagem que armazena imagens em tons de cinza de forma textual legível pelo humano. O formato PGM ASCII é uma variação do formato PGM, que pode armazenar imagens em tons de cinza ou em escala de cinza de forma simples e eficiente.

In [ ]:
for i in range(10):
    # Obtendos as imagens para cada classe
    class_data = train_fmnist[train_fmnist['label'] == i]
    # Armazenando a primeira imagenm como exemplo
    img_example = class_data.values[0]
    """ 0: "Camiseta/top",
    1: "Calça",
    2: "Pulôver",
    3: "Vestido",
    4: "Casaco",
    5: "Sandália",
    6: "Camisa",
    7: "Tênis",
    8: "Bolsa",
    9: "Bota de tornozelo """
    # Criando nome da imagem
    image_name = './pgm_files/image{i}.pgm'
    # Criando imagem
    pgm_file = open(f'./pgm_files/image{i}.pgm','w')
    # Definindo dados do cabeçalho da imagem
    format = 'P2'
    height = '28'
    width = '28'
    max_intensity = '255'

    # Escrevendo cabeçalho
    pgm_file.write(format+'\n')
    pgm_file.write(f'{height} {width}'+'\n')
    pgm_file.write(max_intensity+'\n')

    """ Percorrendo cada linha do arquivo 
    e escrevendo o pixel e adicionando uma quebra 
    de linha a cada 28 pixels """
    height = 28
    for j in range(len(img_example)):
        pixel = str(img_example[j])
        pgm_file.write(pixel + ' ')
        if j == height:
            pgm_file.write('\n')
            height += 28
    
    print(f'File ./pgm_files/image{i}.pgm created!')
    pgm_file.close()

In [ ]:
"""
    read_ascii_pgm
    Lê um arquivo PGM em formato ASCII e retorna a matriz de pixels.
    entrada:
        string com nome do arquivo
    saída:
        vetor bidimensional com dimensões 28x28 com os pixels do arquivo
"""
def read_ascii_pgm(filename):
    
    # Abrindo arquivo em modo de leitura
    with open(filename, 'r') as f:
        # Salavando o conteudo em uma string
        lines = f.readlines()

    """ Obtendo a largura e altura na segunda linha
    separando-as por espaço em branco """
    width, height = map(int, lines[1].split())

    """ Criando um vetor de dimensoes 29x29 para preencher 
    com os pixels """
    data = np.zeros((height+1, width+1), dtype=np.uint8)
    # Ignorando as duas primeiras linhas (formato e dimensões)
    for y in range(3,height):
        # Mapeando os valores para inteiros
        row = map(int, lines[y].split())
        # Percorrendo o vetor e adicionando na matriz de dados
        for x, val in enumerate(row):
            data[y, x] = val
    return data

images = [read_ascii_pgm(f'./pgm_files/image{i}.pgm') for i in range(10)]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
label=0

# Iterando sobre os subplots e imagens correspondentes usando zip(axes.ravel(), images)
# O método zip combina os elementos de axes.ravel() (subplots achatados em uma única dimensão)
# e images (lista de imagens) em pares, permitindo iterar sobre eles ao mesmo tempo
for ax, image in zip(axes.ravel(), images):
    # Mostrando a imagem no subplot atual
    ax.imshow(image, cmap='gray', vmin=0, vmax=255)
    
    # Desativando as bordas do subplot
    ax.axis('off')
    
    # Definindo o título do subplot com base no dicionário de rótulos
    ax.set_title(label_dictionary[label])
    
    # Incrementando o contador de rótulos
    label += 1

# Ajustando o layout da figura
plt.tight_layout()

### **Rede Neural Simples**

In [ ]:
# Importando bibliotecas para criação e treinamento dos modelos
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.regularizers import l2
from keras.utils import plot_model

# Importando bibliotecas para
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ConfusionMatrixDisplay

> - Utilizando a biblioteca TensorFlow com a API Keras para construir e treinar a rede neural


> - A versão 1 do modelo de rede neural possui uma camada Falltten responsável por transformar dados multidimensionais em um vetor unidimensional
> - Além disso, o modelo possui 3 camadas ocultas e uma camada de saída  
> - A camada de entrada possui 784 nós, correspondendo ao número de pixels em cada  imagem
> - As camadas ocultas possuem 200, 100 e 50 neurônios, respectivamente, e usam a função de ativação ReLU
> - A camada de saída possui 10 nós e usa a função de ativação softmax para produzir uma distribuição de probabilidade de pertencimento a cada classe  


> - O modelo é compilado utilizando o otimizador Adam com uma taxa de aprendizado de 0.001  
> - A função de perda escolhida é a entropia cruzada categórica, apropriada para problemas de classificação multiclasse  
> - A métrica de avaliação escolhida é a acurácia

In [ ]:
neural_network_modelv1 = Sequential([
    Flatten(input_shape=(784,)),
    Dense(200, activation='relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])

adam = Adam(learning_rate=0.001)
neural_network_modelv1.compile(optimizer=adam,
              loss=["categorical_crossentropy"],
              metrics=['accuracy'])

neural_network_modelv1.summary()

> - A versão 2 do modelo possui a mesma estrutura da versão 2 mas para fins de redução de **overfitting** o modelo possui uma camada de dropout entre cada camada densa com uma taxa de dropout de 20%  
> - O dropout é uma técnica de regularização que desativa aleatoriamente um determinado percentual de unidades de uma camada durante o treinamento, ajudando a evitar o overfitting 


In [ ]:
neural_network_modelv2 = Sequential([
    Flatten(input_shape=(784,)),
    Dropout(0.2),
    Dense(200, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

adam = Adam(learning_rate=0.001)
neural_network_modelv2.compile(optimizer=adam,
              loss=["categorical_crossentropy"],
              metrics=['accuracy'])

neural_network_modelv2.summary()

#### **Normalização e One-Hot-Encoding**

In [ ]:
# Colunas com os pixels
X_train = train_fmnist.drop('label', axis=1)

# Coluna com a label
Y_train = train_fmnist['label']

In [ ]:
# Normalizando os valores de pixel para o intervalo [0, 1]
X_train = X_train.astype('float32') / 255

> - Dividindo os dados em conjuntos de treinamento e validação (80%, 20%)  
> - A ideia principal de dividir o conjunto de dados em um conjunto de validação é evitar que o modelo se torne bom em classificar as amostras no conjunto de treinamento, mas não seja capaz de generalizar e fazer classificações precisas nos dados que não viu antes

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

> A binarização one-hot encoding permite representar categorias como vetores numéricos, mantendo a distinção entre elas

In [ ]:
label_binarizer = LabelBinarizer()
# Aplicando one-hot encoding nas labels da base de treino
y_train_encoded = label_binarizer.fit_transform(y_train)
# Aplicando one-hot encoding nas labels da base de validação
y_val_encoded = label_binarizer.fit_transform(y_val)

In [ ]:
print("Label original:", y_train.iloc[1])
print("Label após one-hot encoding:", y_train_encoded[1])

### **Treinamento da Rede Neural**

**Treinamento da versão 1 do modelo de rede neural**
> - O número de épocas é definido como 100, sendo assim, a rede será treinada por 100 iterações completas sobre o conjunto de dados de treinamento
> - O tamanho do batch é definido como 500, ous seja, 500 amostras serão usadas em cada passo de treinamento
> - O modo verbose é definido como 1 para que a saída do treinamento será mostrada durante a execução
> - Os dados de treinamento são embaralhados (*shuffle=True*) antes de cada época para evitar que o modelo se ajuste a padrões específicos dos dados de treinamento
> - Além disso, são fornecidos dados de validação (*x_val* e *y_val_encoded*), o que permite avaliar o desempenho do modelo em um conjunto de dados separado durante o treinamento

In [ ]:
neural_network_historyv1 = neural_network_modelv1.fit(np.array(x_train),
          np.array(y_train_encoded),
          epochs=100,
          batch_size=500,
          verbose=1,
          shuffle=True,
          validation_data=(np.array(x_val), np.array(y_val_encoded)))

**Treinamento da versão 2 do modelo de rede neural**
> - O callback *EarlyStopping* serve para interromper o treinamento do modelo automaticamente quando uma determinada condição não é mais atendida, com base em uma métrica monitorada. Neste caso, o callback monitora a perda (loss) no conjunto de treinamento  
> - O parâmetro *patience* indica o número de épocas que o treinamento pode continuar sem melhoria na métrica monitorada antes que o treinamento seja interrompido, sendo assim, se a perda no conjunto de treinamento não mostrar melhoria por 5 épocas consecutivas, o treinamento será interrompido

In [ ]:
callback = EarlyStopping(monitor='loss', patience=5)
neural_network_historyv2 = neural_network_modelv2.fit(np.array(x_train),
          np.array(y_train_encoded),
          epochs=100,
          batch_size=500,
          verbose=1,
          shuffle=True,
          validation_data=(np.array(x_val), np.array(y_val_encoded)),
          callbacks=[callback])

**Visualização das métricas durante o treinamento do modelo**  
O esperado durante o treinamento é que  

- **Curva de acurácia**: deve aumentar ao longo das époas a medida que o modelo é treinado e ajustado para melhorar sua capacidade de fazer previsões corretas no conjunto de treinamento  
- **Curva de perda**: tende a diminuir ao longo das épocas à medida que o modelo é treinado para minimizar a diferença entre as previsões e os rótulos verdadeiros no conjunto de treinamento

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(10,8))
axs[0, 0].plot(neural_network_historyv1.history['accuracy'], color='violet', label='accuracy')
axs[0, 0].plot(neural_network_historyv1.history['val_accuracy'], color='darkviolet', label='val_accuracy')
axs[0, 0].set_title('Neural Network Accuracy v1', fontsize=10)

axs[0, 1].plot(neural_network_historyv1.history['loss'], color='orange',label='loss')
axs[0, 1].plot(neural_network_historyv1.history['val_loss'], color='tomato', label='val_loss')
axs[0, 1].set_title('Neural Network Loss v1', fontsize=10)

axs[0, 0].legend(loc="upper left")
axs[0, 1].legend(loc="upper left")

axs[1, 0].plot(neural_network_historyv2.history['accuracy'], color='violet', label='accuracy')
axs[1, 0].plot(neural_network_historyv2.history['val_accuracy'], color='darkviolet', label='val_accuracy')
axs[1, 0].set_title('Neural Network Accuracy v2', fontsize=10)

axs[1, 1].plot(neural_network_historyv2.history['loss'], color='orange',label='loss')
axs[1, 1].plot(neural_network_historyv2.history['val_loss'], color='tomato', label='val_loss')
axs[1, 1].set_title('Neural Network Loss v2', fontsize=10)

axs[1, 0].legend(loc="upper left")
axs[1, 1].legend(loc="upper left")
plt.tight_layout()
plt.show()

# Overfitting
# 100 epocas
# Early stop
# Dropout 20%

> - Os resultados do modelo v1, demonstram que a curva de acurácia para os dados de validação se mantém constante enquanto a curva de perda de validação aumenta, isso indica um caso de **overfitting**, onde o modelo está se ajustando muito bem aos dados de treinamento, mas não está generalizando bem para dados não vistos  
> - Sendo assim, para melhorar o desempenho da versão 1 do modelo, foram implementadas as técnicas de Dropout na modelagem e de Early Stopping para a versão 2 do modelo e os resultados obtidos foram significativamente melhores

### **Desempenho da Rede Neural nos Dados de Teste**

In [ ]:
# Carregando os dados de teste em um DataFrame
test_fminist = pd.read_csv('./dataset/fashion-mnist_test.csv')
test_fminist.shape

In [ ]:
# Colunas com os pixels
X_test = test_fminist.drop('label', axis=1)

# Coluna com a label
Y_test = test_fminist['label']

In [ ]:
# Normalizando os valores para estarem no intervalo [0,1]
x_test = X_test.astype('float32') / 255

In [ ]:
# Aplicando one-hot encoding nas labels de teste
y_test_encoded = label_binarizer.fit_transform(Y_test)

In [ ]:
results = neural_network_modelv1.evaluate(np.array(x_test), y_test_encoded, batch_size=128)

print('Test loss:', results[0])
print('Test accuracy:', results[1])

In [ ]:
results = neural_network_modelv2.evaluate(np.array(x_test), y_test_encoded, batch_size=128)

print('Test loss:', results[0])
print('Test accuracy:', results[1])

> Os valores de acurácia para a versão 2 do modelo foi suavemente melhor do que para a versão 1, contudo, a queda no valor de perda foi significativa o que demonstra que o modelo melhorou em termos de generalização. Para ter uma conclusão mais precisa, vamos avaliar três outras métricas:
- Precisão 
- Recall
- F1-score

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

In [ ]:
y_true = Y_test
y_pred = neural_network_modelv1.predict(x_test)
y_pred_discrete = np.argmax(y_pred, axis=1)

precision_v1, recall_v1, f1_score_v1, _ =precision_recall_fscore_support(y_true, y_pred_discrete, average='macro')

y_pred = neural_network_modelv1.predict(x_test)
y_pred_discrete = np.argmax(y_pred, axis=1)
precision_v2, recall_v2, f1_score_v2, _ =precision_recall_fscore_support(y_true, y_pred_discrete, average='macro')

#### **Matriz de Confusão para Avaliação dos Resultados**

In [ ]:
class_names = list(label_dictionary.values())
print(class_names)

In [ ]:
class estimator:
  _estimator_type = ''
  classes_=[]
  def __init__(self, model, classes):
    self.model = model
    self._estimator_type = 'classifier'
    self.classes_ = classes
  def predict(self, X):
    y_prob= self.model.predict(X)
    y_pred = y_prob.argmax(axis=1)
    return y_pred

In [ ]:
classifier = estimator(model, class_names)
figsize = (9,9)
ConfusionMatrixDisplay.from_estimator(classifier,X_test,Y_test, 
                                      cmap = 'Purples', 
                                      normalize='true', 
                                      ax=plt.subplots(figsize=figsize)[1],
                                      display_labels=class_names,
                                      xticks_rotation=45)
plt.show()

### **Rede Neural Convolucional (CNN)**

In [ ]:
# Colunas dos pixels
X_train = train_fmnist.drop('label', axis=1)

# Colunas com a label
Y_train = train_fmnist['label']

In [ ]:
x_train_cnn = np.array(X_train)
x_train_cnn = x_train_cnn.reshape((x_train_cnn.shape[0], 28, 28, 1)).astype('float32') / 255
x_train_cnn.shape

In [ ]:
x_train_cnn, x_val_cnn, y_train_cnn, y_val_cnn = train_test_split(x_train_cnn, Y_train, test_size=0.2, random_state=42)

In [ ]:
label_binarizer = LabelBinarizer()
y_train_encoded_cnn = label_binarizer.fit_transform(y_train_cnn)
y_val_encoded_cnn = label_binarizer.fit_transform(y_val_cnn)

> A utilização de convoluções seguidas por operações de pooling permite que o modelo capture características locais e espaciais nas imagens, tornando-o adequado para tarefas de classificação de imagens.

In [ ]:
model_cnn = Sequential([
    Conv2D(32, (3,3), 1, activation='relu', 
        input_shape=(28,28,1),
        kernel_regularizer=l2(0.01)),
    MaxPooling2D(),

    Conv2D(64, (3,3), 1, activation='relu',
        kernel_regularizer=l2(0.01)),
    MaxPooling2D(),

    Conv2D(128, (3,3), 1, activation='relu',
        kernel_regularizer=l2(0.01)),
    MaxPooling2D(),

    Flatten(),
    
    Dense(512, activation='relu',
          kernel_regularizer=l2(0.01)),
    
    Dense(10, activation='softmax',
        kernel_regularizer=l2(0.01))
])

adam = Adam(learning_rate=0.001)
model_cnn.compile(optimizer=adam,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])
""" sgd_optimizer = SGD(learning_rate=0.001)
model_cnn.compile(optimizer=sgd_optimizer, loss="categorical_crossentropy", metrics=['accuracy']) """

model_cnn.summary()

In [ ]:
# plot_model(model_cnn, show_shapes=True, show_layer_names=True, dpi=72)
plot_model(model_cnn, show_shapes=True, show_layer_names=True, dpi=72, to_file='./cnn_model.png')

### **Treinamento da Rede Neural Convolucional**

In [ ]:
cnn_callback = EarlyStopping(monitor='loss', patience=3)
cnn_history = model_cnn.fit(x_train_cnn,
            y_train_encoded_cnn,
            epochs=100,
            batch_size=500,
            verbose=1,
            shuffle=True,
            validation_data=(x_val_cnn, y_val_encoded_cnn),
            callbacks=[cnn_callback])

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,5))

axs[0].plot(cnn_history.history['accuracy'], color='violet', label='accuracy')
axs[0].plot(cnn_history.history['val_accuracy'], color='darkviolet', label='val_accuracy')
axs[0].set_title('CNN Accuracy', fontsize=10)

axs[1].plot(cnn_history.history['loss'], color='orange',label='loss')
axs[1].plot(cnn_history.history['val_loss'], color='tomato', label='val_loss')
axs[1].set_title('CNN Loss', fontsize=10)

axs[0].legend(loc="upper left")
axs[1].legend(loc="upper left")
plt.show()

### **Desempenho da CNN nos Dados de Teste**

In [ ]:
x_test_cnn = np.array(x_test).reshape((x_test.shape[0], 28, 28, 1))
y_test_encoded = label_binarizer.fit_transform(Y_test)

In [ ]:
results = model_cnn.evaluate(x_test_cnn, y_test_encoded)

print('Test loss:', results[0])
print('Test accuracy:', results[1])

#### **Matriz de Confusão para Avaliação dos Resultados**

In [ ]:
classifier = estimator(model_cnn, class_names)
figsize = (9,9)
ConfusionMatrixDisplay.from_estimator(classifier,x_test_cnn,Y_test, 
                                      cmap = 'Purples', 
                                      normalize='true', 
                                      ax=plt.subplots(figsize=figsize)[1],
                                      display_labels=class_names,
                                      xticks_rotation=45)
plt.show()